In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings("ignore")

import tqdm
pd.set_option("display.max_columns",None)

In [ ]:
data2 = pd.read_csv('merge_data.csv',index_col=0)

In [ ]:
# import package
from sklearn.preprocessing import StandardScaler,MinMaxScaler

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet

from sklearn.metrics import r2_score

In [ ]:
# GBR Best parameters
import numpy as np
from sklearn import linear_model
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import GradientBoostingRegressor as GBR

# store r2_score
rList=[]
combineList = []

#parameters
loss_list = ['huber'] #, 'squared_error']
learning_rate = [0.01, 0.03, 0.05, 0.1]
max_depth = [3,4,5,6]

for i in range(30):
    df_train = data2[(data2.yyyymm>=195703)&(data2.yyyymm<197503)]
    df_validation = data2[(data2.yyyymm>=197503)&(data2.yyyymm<198703)]
    df_test = data2[(data2.yyyymm>=198703)&(data2.yyyymm<201603)]

    df_train = df_train.copy()
    df_validation = df_validation.copy()
    df_test = df_test.copy()

    trainingstart = 195703
    trainingend = 197503 + i*100
    validend = trainingend + 1200
    testend = validend + 100

    trainingMask = (data2.yyyymm >= trainingstart) & (data2.yyyymm< trainingend)
    trainingData = data2.loc[trainingMask]
    
    validationMask = (data2.yyyymm >= trainingend) & (data2.yyyymm< validend)
    validationData = data2.loc[validationMask]
    
    testMask = (data2.yyyymm >= validend) & (data2.yyyymm < testend)
    testData = data2.loc[testMask]
    
    
    trainingData_y = trainingData['excess_ret']
    trainingData_x = trainingData.drop(['excess_ret','permno', 'yyyymm'], axis=1)
    
    validationData_y = validationData['excess_ret']
    validationData_x = validationData.drop(['excess_ret','permno', 'yyyymm'], axis=1)
    
    testData_y = testData['excess_ret']
    testData_x = testData.drop(['excess_ret','permno', 'yyyymm'], axis =1)
    
    bestRSqr = float("-inf")
    bestGBR = None
    bestCombine = None
    
    for lr in learning_rate:
        for max_dep in max_depth:
            gbr = GBR(max_depth=max_dep,learning_rate=lr,loss=l)
            gbr.fit(X_train,y_train)

            pred = gbr.predict(validationData_x)
            currentRSqr = r2_score(validationData_y.values, pred)
            
            if(currentRSqr>bestRSqr):
                bestRSqr=currentRSqr
                bestGBR=gbr
                bestCombine=(learning_rate,max_depth)
        
    resultPred = bestEnet.predict(testData_x)
    resultRSqr = r2_score(testData_y.values, resultPred)
    print('result R square for batch',i,': ', resultRSqr, 'best combine:', bestCombine)
    rList.append(resultRSqr)
    combineList.append(bestCombine)

In [ ]:
#The recursive performance evaluation scheme
train_score2=[0]*30
validation_score2=[0]*30
test_score2=[0]*30
scaler=StandardScaler()
minmax=MinMaxScaler()

In [ ]:
from sklearn.tree import export_graphviz
from sklearn.ensemble import GradientBoostingRegressor as GBR

bestRSqr = float("-inf")
bestGBR = None
loss_list = ['huber']
max_dep = 3
lr = 0.01
best_year = 0

for i in range(0,30):
    print('Batch {}:\nTrain data:1960-{}\nValuation data:{}-{}\nTest data:{}-{}'.format(i,1978+i,1978+i,1990+i,1990+i,1991+i))
    data_train=data2[(data2.yyyymm>=196001)&(data2.yyyymm<197801+i*100)]
    data_validation=data2[(data2.yyyymm>=197801+i*100)&(data2.yyyymm<199001+i*100)]
    data_test=data2[(data2.yyyymm>=196001+i*100)&(data2.yyyymm<196101+i*100)]

    #Normalize
    X_train=data_train.drop("excess_ret",axis=1).copy()
    X_train=scaler.fit_transform(X_train)
    X_validation=data_validation.drop("excess_ret",axis=1).copy()
    X_validation=scaler.transform(X_validation)
    X_test=data_test.drop("excess_ret",axis=1).copy()
    X_test=scaler.transform(X_test)

    y_train=data_train[["excess_ret"]].copy()
    y_validation=data_validation[["excess_ret"]].copy()
    y_test=data_test[["excess_ret"]].copy()
    y_train=scaler.fit_transform(y_train)
    y_test=scaler.transform(y_test)
    y_validation=scaler.transform(y_validation)
    
    gbr = GBR(max_depth=3,learning_rate=0.01,loss='huber')
    gbr.fit(X_train,y_train)
    y_train_pred=gbr.predict(X_train)
    train_score2[i]=r2_score(y_train,y_train_pred)
    y_validation_pred=gbr.predict(X_validation)
    validation_score2[i]=r2_score(y_validation,y_validation_pred)
    y_test_pred=gbr.predict(X_test)
    test_score2[i]=r2_score(y_test,y_test_pred)
    currentRSqr = test_score2[i]

    if(currentRSqr>bestRSqr):
        bestRSqr=currentRSqr
        bestRFR = gbr
        best_year = 196001+i*100

In [ ]:
test_score =pd.DataFrame(test_score2)
print(test_score)

In [ ]:
# Performence of GBR+H
plt.figure(figsize=(10,5))
plt.title("GBR Performance")
year=np.arange(1990,2020)
plt.plot(year,test_score2)
plt.xlabel("year")
plt.ylabel("score--R-square")

In [ ]:
#The Importance 
#X_test0 = best_data1.drop(best_data1.columns[0],axis=1,inplace=True).copy()

X_test0 = data2[(data2.yyyymm>=best_year)&(data2.yyyymm<best_year+100)]
print(X_test0.shape)
a = X_test0.drop("excess_ret",axis=1).copy()
print(a.shape)
id = a.columns
L = len(a.columns)
train_score=[0]*L
validation_score=[0]*L
test_score2 = [0]*L
importance_GBR_H=[0]*L
L

In [ ]:
#Variable importance

for i in range(0,L):
    
    X_test = X_test0.drop("excess_ret",axis=1).copy()
    X_test.loc[:,id[i]]=0
    X_test=scaler.fit_transform(X_test)

    y_test=X_test0[["excess_ret"]].copy()
    y_test=scaler.fit_transform(y_test)
    

    y_test_pred=bestRFR.predict(X_test)
    test_score2[i] = r2_score(y_test,y_test_pred)
    importance_GBR_H[i] = bestRSqr - test_score2[i]

    
    del X_test,y_test,

In [ ]:
importance_GBR_H = np.abs(importance_GBR_H)
importance_GBR_H1 = pd.DataFrame(importance_GBR_H,columns=['Importance'], index=id)
importance_GBR_H1 = importance_GBR_H1.sort_values(by='Importance',ascending=True)
importance_GBR_H1 = importance_GBR_H1.tail(20)
importance_GBR_H1.plot(kind='barh', figsize=(9, 7))